# Install the New Worker- BETA!!
### Remember to first set your API KEY and worker name
#### You can serve different models, simply change the name in models_to_load to match the model you want, you can check either https://aqualxx.github.io/stable-ui/workers, in the models tab, or https://tinybots.net/artbot/info/models, just copy paste the name (it's set to Deliberate by default)
#### You can also change max_power and see how high you can go, it's set to 20 by default

In [ ]:
#0.- This cell will set the variables and rerun the worker if you stopped it, but only if everything else was installed already

#### For right now, THESE are the only variables that we care about
worker_name = ""
api_key = ""
max_power = 20
safety_on_gpu = True
models_to_load = ["Deliberate"]
allow_lora = True
allow_controlnet = True
nsfw = True
censor_nsfw = False
####For right now, THESE are the only variables that we care about


queue_size = 1
max_threads = 1
horde_url = "https://aihorde.net"
allow_painting = False
dynamic_models = False
models_to_skip = ["stable_diffusion_inpainting", "stable_diffusion_2.1",  "stable_diffusion_2.0"]
allow_post_processing = False
priority_usernames = []
blacklist = []
censorlist = []
allow_img2img = True
allow_unsafe_ip = True
number_of_dynamic_models = 0
max_models_to_download = 10
forms = ["caption","nsfw","interrogation","post-process"]


current_path = "/content/"
worker_path = current_path + "horde-worker-reGen/"
bridgeData_file = worker_path + "bridgeData.yaml"

import os

if not os.path.exists(bridgeData_file):
    print ("bridgeData.yaml file not found. Proceed to install the worker.")
else:
    print ("bridgeData.yaml file found. Recreating bridgeData.yaml and restarting the worker.")
    create_yaml()
    !cd /content
    !source ../regen/bin/activate;python download_models.py
    !cd /content
    !source ../regen/bin/activate;python run_worker.py

In [ ]:
#1.- Virtual Environment

!apt-get update
!apt install python3.10-venv
!python -m venv regen

In [ ]:
#2.- Remove the worker if it exists, then, Clone the regen worker

!cd /content;rm -r horde-worker-reGen
!cd /content;git clone https://github.com/Haidra-Org/horde-worker-reGen.git

In [ ]:
#3.- Install requirements for cuda 11.8

!source regen/bin/activate;pip install -r .\/horde-worker-reGen/requirements.118.txt

In [ ]:
#4.- Work around so hordelib installs @ cuda 11.8

%cd /content/

!source regen/bin/activate;pip install -r .\/horde-worker-reGen/requirements.hordelib.txt

!source regen/bin/activate;pip install hordelib --no-deps

In [ ]:
#5.- Create .yaml config file


import os

%cd $worker_path
print ("Creating bridgeData.yaml file.")

def create_yaml():

    from yaml import load, dump

    def make_yaml_sublist(list_to_convert: list[str]):
        sublist_yaml = dump(list_to_convert)
        sublist_yaml = "\n" + sublist_yaml
        return sublist_yaml



    data = f"""horde_url: "{horde_url}"
api_key: "{api_key}"
priority_usernames: []
max_threads: {max_threads}
queue_size: {queue_size}
safety_on_gpu: {safety_on_gpu}
require_upfront_kudos: false
dreamer_name: "{worker_name}"
max_power: {max_power}
nsfw: {nsfw.__str__().lower()}
censor_nsfw: {censor_nsfw}
blacklist: {blacklist}
censorlist: {censorlist}
allow_img2img: {allow_img2img.__str__().lower()}
allow_painting: {allow_painting.__str__().lower()}
allow_unsafe_ip: true
allow_post_processing: {allow_post_processing.__str__().lower()}
allow_controlnet: {allow_controlnet.__str__().lower()}
allow_lora: {allow_lora.__str__().lower()}
max_lora_cache_size: 10
dynamic_models: false
number_of_dynamic_models: 0
max_models_to_download: 10
stats_output_frequency: 30
cache_home: "./"
always_download: true
temp_dir: "./tmp"
disable_terminal_ui: True
vram_to_leave_free: "80%"
ram_to_leave_free: "80%"
disable_disk_cache: false
models_to_load: {make_yaml_sublist(models_to_load)}
models_to_skip: {make_yaml_sublist(models_to_skip)}
suppress_speed_warnings: false
forms:
- "caption"
- "nsfw"
- "interrogation"
- "post-process"
"""

    with open(bridgeData_file, "w") as text_file:
        text_file.write(data)

    print ("bridgeData.yaml file created.")

create_yaml()

In [ ]:
#6.- Run download models
# Make sure you have the correct path based on any `cd` commands above
!cd /content

!source ../regen/bin/activate;python download_models.py

In [ ]:
#7.- Run the worker
# Make sure you have the correct path based on any `cd` commands above
!cd /content

!source ../regen/bin/activate;python run_worker.py